In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load the portfolio datasets
dow_jones_df = pd.read_csv('Dow_Jones.csv')
nasdaq_df = pd.read_csv('NASDAQ .csv')
portfolio_df = pd.read_csv('Portfolio.csv')
portfolio_prices_df = pd.read_csv('Portfolio_prices.csv')
sp500_df = pd.read_csv('SP500.csv')

print("Dataset shapes:")
print(f"Dow Jones: {dow_jones_df.shape}")
print(f"NASDAQ: {nasdaq_df.shape}")
print(f"Portfolio: {portfolio_df.shape}")
print(f"Portfolio Prices: {portfolio_prices_df.shape}")
print(f"S&P 500: {sp500_df.shape}")

print("\nColumn structures:")
print("Portfolio columns:", portfolio_df.columns.tolist())
print("Portfolio Prices columns:", portfolio_prices_df.columns.tolist())
print("S&P 500 columns:", sp500_df.columns.tolist())

# Initial data inspection
print("\nPortfolio data sample:")
print(portfolio_df.head())

print("\nPortfolio prices sample:")
print(portfolio_prices_df.head())

print("\nS&P 500 data sample:")
print(sp500_df.head())


Dataset shapes:
Dow Jones: (1312, 9)
NASDAQ: (1312, 9)
Portfolio: (27, 5)
Portfolio Prices: (35397, 9)
S&P 500: (1312, 9)

Column structures:
Portfolio columns: ['Ticker', 'Quantity', 'Sector', 'Close', 'Weight']
Portfolio Prices columns: ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adjusted', 'Returns', 'Volume']
S&P 500 columns: ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adjusted', 'Returns', 'Volume']

Portfolio data sample:
  Ticker  Quantity                Sector       Close    Weight
0   AAPL         1                    IT  218.270004  3.461290
1   ADAP         1       Pharmaceuticals    0.291000  0.004615
2   AGCO         1           Agriculture   93.089996  1.476206
3     BA         2  Military Engineering  178.110001  2.824439
4     BG         4           Agriculture   73.250000  1.161586

Portfolio prices sample:
         Date Ticker        Open        High         Low       Close  \
0  2020-01-03    JPM  137.500000  139.229996  137.080002  138.339996   
1  2

In [ ]:
# Step 2: Complete Portfolio Analysis
portfolio_weights = dict(zip(portfolio_df['Ticker'], portfolio_df['Weight'] / 100))
portfolio_data = portfolio_prices_df[portfolio_prices_df['Ticker'].isin(portfolio_weights.keys())].copy()
portfolio_data['Weight'] = portfolio_data['Ticker'].map(portfolio_weights)
portfolio_data['Weighted_Return'] = portfolio_data['Returns'].fillna(0) * portfolio_data['Weight']
portfolio_returns = portfolio_data.groupby('Date')['Weighted_Return'].sum()

In [ ]:
annual_return = portfolio_returns.mean() * 252
volatility = portfolio_returns.std() * np.sqrt(252)
sharpe_ratio = annual_return / volatility
sp500_returns = sp500_df.set_index('Date')['Returns'].reindex(portfolio_returns.index).fillna(0)
beta = portfolio_returns.cov(sp500_returns) / sp500_returns.var()
sector_weights = portfolio_df.groupby('Sector')['Weight'].sum()
var_95 = np.percentile(portfolio_returns, 5)
cumulative = (1 + portfolio_returns).cumprod()
max_drawdown = ((cumulative - cumulative.cummax()) / cumulative.cummax()).min()

In [ ]:
# Step 3: Export for agents
import json
agent_data = {
    'performance': {'return': float(annual_return), 'sharpe': float(sharpe_ratio), 'beta': float(beta)},
    'sectors': sector_weights.to_dict(),
    'risk': {'var': float(var_95), 'drawdown': float(max_drawdown)}
}

with open('financial_agent_insights.json', 'w') as f:
    json.dump(agent_data, f, indent=2)

print("Analysis complete. Ready for Langflow.")

Analysis complete. Ready for Langflow.
